In [1]:
#Altimetrias.net usa un código de tres dígitos para identificar los puertos, del 001 al 766.

base_url_puerto = 'https://www.altimetrias.net/aspbk/verPuerto.asp?id='

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
puerto = '001'

url = f'{base_url_puerto}{puerto}'

In [4]:
html = requests.get(url).text

In [5]:
soup = BeautifulSoup(html, 'html.parser')

In [6]:
#En esta tabla encontramos la provincia, nombre del puerto y localidad:
tabla_0 = soup.find_all('table')[0]
tabla_0

<table align="center" bgcolor="#000000" cellpadding="5" cellspacing="1" width="100%">
<tr>
<td align="center" id="provincias" nowrap="" width="15%">
<img border="0" height="84" src="imagenes/4Bizkaia.png" width="112"/><font color="#F97771">BIZKAIA</font></td>
<td align="center" width="75%"><font color="#FFCC00" size="6"><b>NABARNIZ</b></font><br/><font color="#FFFFFF" size="5"><b>Aulesti-Narea</b></font><!-- <a class="LinkGorri" href="../asppdf/importare/importare.asp?id=1&nombre=Nabarniz">&nbsp;</a> --></td>
<td align="center" width="15%"><a href="../default.asp"><img border="0" src="../imagenes/verPuerto.jpg"/></a><font color="#FFFF00" size="3"><b>15706</b></font> <font color="#C0C0C0" size="3"><b>visitas</b></font></td>
</tr>
</table>

In [7]:
#En esta tabla veremos altitud, distancia, desnivel, pendiente y coeficiente.
tabla_1 = soup.find_all('table')[1]
tabla_1

<table align="center" bgcolor="#000000" cellpadding="5" cellspacing="1" width="100%">
<tr align="center" bgcolor="#DDDDDD">
<td nowrap="" width="1%"><img alt="" border="0" height="25" src="imagenes/xaltitud.png" width="25"/> <font size="3"><b>Altitud: </b>365 m</font></td>
<td nowrap="" width="1%"><img alt="" border="0" height="25" src="imagenes/xdistancia.png" width="25"/> <font size="3"><b>Distancia:</b> 4,4 km</font></td>
<td nowrap="" width="1%"><img alt="" border="0" height="25" src="imagenes/xdesnivel.png" width="25"/> <font size="3"><b>Desnivel:</b> 273 m</font></td>
<td nowrap="" width="1%"><img alt="" border="0" height="25" src="imagenes/xpendiente.png" width="25"/> <font size="3"><b>Pendiente Media: </b>6,2 %</font></td>
<td nowrap="" width="1%"><font size="3"><b>Coeficiente: </b><font color="red"><b>81</b></font></font></td>
</tr>
</table>

In [8]:
filas_0 = tabla_0.find_all('td')

In [9]:
#Provincia.

filas_0[0] 

<td align="center" id="provincias" nowrap="" width="15%">
<img border="0" height="84" src="imagenes/4Bizkaia.png" width="112"/><font color="#F97771">BIZKAIA</font></td>

In [10]:
#Población y puerto.

filas_0[1] 

<td align="center" width="75%"><font color="#FFCC00" size="6"><b>NABARNIZ</b></font><br/><font color="#FFFFFF" size="5"><b>Aulesti-Narea</b></font><!-- <a class="LinkGorri" href="../asppdf/importare/importare.asp?id=1&nombre=Nabarniz">&nbsp;</a> --></td>

In [11]:
#Obteniendo la provincia:

filas_0[0].text.strip()

'BIZKAIA'

In [12]:
#Ahora el puerto:

filas_0[1].contents[0].text.strip()

'NABARNIZ'

In [13]:
#Y finalmente la población:

filas_0[1].contents[2].text.strip()

'Aulesti-Narea'

In [14]:
#Ahora vamos a seguir el mismo procedimiento para las cifras del puerto.

filas_1 = tabla_1.find_all('td')

In [15]:
#Altitud:
filas_1[0].contents[2].text.strip()

'Altitud: 365 m'

In [16]:
#Distancia:
filas_1[1].contents[2].text.strip()

'Distancia: 4,4 km'

In [17]:
#Desnivel:
filas_1[2].contents[2].text.strip()

'Desnivel: 273 m'

In [18]:
#Pendiente:
filas_1[3].contents[2].text.strip()

'Pendiente Media: 6,2 %'

In [19]:
#Coeficiente:
filas_1[4].contents[0].text.strip()

'Coeficiente: 81'

In [20]:
#Creando un diccionario del puerto:
import re

puerto = [{'puerto': filas_0[1].contents[0].text.strip(),
          'provincia': filas_0[0].text.strip(),
          'pueblo': filas_0[1].contents[2].text.strip(),
          'altitud': re.findall(r"\d+", filas_1[0].contents[2].text.strip())[0],
          'distancia': re.findall(r"\d+,\d", filas_1[1].contents[2].text.strip())[0],
          'desnivel': re.findall(r"\d+",filas_1[2].contents[2].text.strip())[0],
          'pendiente': re.findall(r"\d+,\d",filas_1[3].contents[2].text.strip())[0],
          'coeficiente': re.findall(r"\d+",filas_1[4].contents[0].text.strip())[0]}]

In [21]:
puerto

[{'puerto': 'NABARNIZ',
  'provincia': 'BIZKAIA',
  'pueblo': 'Aulesti-Narea',
  'altitud': '365',
  'distancia': '4,4',
  'desnivel': '273',
  'pendiente': '6,2',
  'coeficiente': '81'}]

In [22]:
#Definiendo una función para extraer el diccionario de cada puerto:  

def scraper_puertos(x):
    if len(str(x)) == 1:
        x = '00' + str(x)
    elif len(str(x)) == 2:
        x = '0' + str(x)
    else:
        pass
    base_url_puerto = 'https://www.altimetrias.net/aspbk/verPuerto.asp?id='
    url = f'{base_url_puerto}{x}'
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    tabla_0 = soup.find_all('table')[0]
    tabla_1 = soup.find_all('table')[1]
    filas_0 = tabla_0.find_all('td')
    filas_1 = tabla_1.find_all('td')
    puerto = [{'puerto': filas_0[1].contents[0].text.strip(),
          'provincia': filas_0[0].text.strip(),
          'pueblo': filas_0[1].contents[2].text.strip(),
          'altitud': int(re.findall(r"\d+", filas_1[0].contents[2].text.strip())[0]),
          'distancia': int(re.findall(r"\d+", filas_1[1].contents[2].text.strip())[0]),
          'desnivel': int(re.findall(r"\d+",filas_1[2].contents[2].text.strip())[0]),
          'pendiente': int(re.findall(r"\d+",filas_1[3].contents[2].text.strip())[0]),
          'coeficiente': int(re.findall(r"\d+",filas_1[4].contents[0].text.strip())[0])}]
    return puerto
    print(puerto)

In [23]:
for i in range(1,5):
    try:
        puerto_gen = scraper_puertos(i)
        print(puerto_gen)
    except:
        pass  

[{'puerto': 'NABARNIZ', 'provincia': 'BIZKAIA', 'pueblo': 'Aulesti-Narea', 'altitud': 365, 'distancia': 4, 'desnivel': 273, 'pendiente': 6, 'coeficiente': 81}]
[{'puerto': 'ANDRAKA', 'provincia': 'BIZKAIA', 'pueblo': 'Plentzia', 'altitud': 138, 'distancia': 3, 'desnivel': 129, 'pendiente': 3, 'coeficiente': 17}]
[{'puerto': 'ANDRAKA', 'provincia': 'BIZKAIA', 'pueblo': 'Asteintza (Jatabe/Maruri)', 'altitud': 138, 'distancia': 3, 'desnivel': 114, 'pendiente': 3, 'coeficiente': 15}]
[{'puerto': 'ARTEBAKARRA', 'provincia': 'BIZKAIA', 'pueblo': 'Derio', 'altitud': 137, 'distancia': 3, 'desnivel': 113, 'pendiente': 3, 'coeficiente': 15}]


In [24]:
import pandas as pd

df_puertos = pd.DataFrame.from_dict(scraper_puertos(1))
df_puertos.head()

,puerto,provincia,pueblo,altitud,distancia,desnivel,pendiente,coeficiente
0,NABARNIZ,BIZKAIA,Aulesti-Narea,365,4,273,6,81


In [25]:
for i in range(2,766):
    try:
        puerto_gen = scraper_puertos(i)
        df_puertos = df_puertos.append(puerto_gen, ignore_index=True)
    except:
        pass 

In [26]:
df_puertos.head()

df_puertos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   puerto       745 non-null    object
 1   provincia    745 non-null    object
 2   pueblo       745 non-null    object
 3   altitud      745 non-null    int64 
 4   distancia    745 non-null    int64 
 5   desnivel     745 non-null    int64 
 6   pendiente    745 non-null    int64 
 7   coeficiente  745 non-null    int64 
dtypes: int64(5), object(3)
memory usage: 46.7+ KB


In [27]:
df_puertos.groupby('provincia').count().sort_values('puerto', ascending=False).head(5)

,puerto,pueblo,altitud,distancia,desnivel,pendiente,coeficiente
provincia,,,,,,,
BIZKAIA,159,159,159,159,159,159,159
CANTABRIA,57,57,57,57,57,57,57
GIPUZKOA,54,54,54,54,54,54,54
BURGOS,44,44,44,44,44,44,44
NAVARRA,37,37,37,37,37,37,37


In [28]:
df_puertos.sort_values('coeficiente', ascending=False, inplace=True)

In [29]:
df_puertos.head()

,puerto,provincia,pueblo,altitud,distancia,desnivel,pendiente,coeficiente
696,SIERRA NEVADA-PICO VELETA,GRANADA,Haza Llanas-Las Sabinas,3296,39,2527,6,616
386,SIERRA NEVADA-PICO VELETA,GRANADA,Güéjar Sierra,3367,44,2557,5,549
250,ANGLIRU,ASTURIAS,Santa Eulalia,1570,18,1423,7,528
267,ANGLIRU,ASTURIAS,La Vega (Riosa),1572,12,1266,10,515
452,GAMONITEIRO,ASTURIAS,Pola-Cobertoria,1772,15,1465,9,492


In [30]:
df_puertos.drop_duplicates(subset ="puerto", inplace = True)
df_puertos.head()

,puerto,provincia,pueblo,altitud,distancia,desnivel,pendiente,coeficiente
696,SIERRA NEVADA-PICO VELETA,GRANADA,Haza Llanas-Las Sabinas,3296,39,2527,6,616
250,ANGLIRU,ASTURIAS,Santa Eulalia,1570,18,1423,7,528
452,GAMONITEIRO,ASTURIAS,Pola-Cobertoria,1772,15,1465,9,492
402,ROQUE DE LOS MUCHACHOS,SC DE TENERIFE,Garafía,2400,29,2164,7,477
148,TEIDE,SC DE TENERIFE,Santa Cruz,2325,63,2308,3,474


In [31]:
df_coef_provincia = df_puertos.groupby('provincia').mean('coeficiente')

In [32]:
df_coef_provincia.head()

,altitud,distancia,desnivel,pendiente,coeficiente
provincia,,,,,
A CORUÑA,505.80000,6.600000,436.000000,7.000000,117.20000
ALBACETE,1402.50000,12.500000,782.500000,6.000000,174.50000
ALICANTE,810.87500,7.750000,577.625000,7.750000,171.12500
ALMERÍA,1670.87500,18.750000,1001.375000,4.875000,216.37500
ASTURIAS,1163.37931,11.551724,818.068966,7.034483,226.62069


In [33]:
df_coef_provincia.sort_values('coeficiente', ascending=False).head(10)

,altitud,distancia,desnivel,pendiente,coeficiente
provincia,,,,,
LAS PALMAS,1282.333333,16.666667,1154.000000,6.000000,293.666667
SC DE TENERIFE,1320.142857,21.000000,1117.714286,6.428571,246.428571
ASTURIAS,1163.379310,11.551724,818.068966,7.034483,226.620690
ALMERÍA,1670.875000,18.750000,1001.375000,4.875000,216.375000
OURENSE,1386.666667,22.000000,1067.333333,3.666667,212.000000
JAÉN,1464.000000,12.750000,838.000000,6.250000,210.500000
GRANADA,1374.700000,14.500000,853.400000,5.750000,204.000000
GERONA,1284.000000,13.000000,807.250000,6.000000,191.500000
MURCIA,823.250000,10.500000,712.000000,6.750000,181.500000


In [34]:
df__coef_provincia = df_coef_provincia.drop('LAS PALMAS', inplace=True)

In [35]:
df_coef_provincia.sort_values('coeficiente', ascending=False).head(10)

,altitud,distancia,desnivel,pendiente,coeficiente
provincia,,,,,
SC DE TENERIFE,1320.142857,21.000000,1117.714286,6.428571,246.428571
ASTURIAS,1163.379310,11.551724,818.068966,7.034483,226.620690
ALMERÍA,1670.875000,18.750000,1001.375000,4.875000,216.375000
OURENSE,1386.666667,22.000000,1067.333333,3.666667,212.000000
JAÉN,1464.000000,12.750000,838.000000,6.250000,210.500000
GRANADA,1374.700000,14.500000,853.400000,5.750000,204.000000
GERONA,1284.000000,13.000000,807.250000,6.000000,191.500000
MURCIA,823.250000,10.500000,712.000000,6.750000,181.500000
LUGO,1210.600000,19.800000,780.600000,3.600000,177.400000


In [36]:
df__coef_provincia = df_coef_provincia.drop('SC DE TENERIFE', inplace=True)
df_coef_provincia.sort_values('coeficiente', ascending=False).head(10)

,altitud,distancia,desnivel,pendiente,coeficiente
provincia,,,,,
ASTURIAS,1163.379310,11.551724,818.068966,7.034483,226.62069
ALMERÍA,1670.875000,18.750000,1001.375000,4.875000,216.37500
OURENSE,1386.666667,22.000000,1067.333333,3.666667,212.00000
JAÉN,1464.000000,12.750000,838.000000,6.250000,210.50000
GRANADA,1374.700000,14.500000,853.400000,5.750000,204.00000
GERONA,1284.000000,13.000000,807.250000,6.000000,191.50000
MURCIA,823.250000,10.500000,712.000000,6.750000,181.50000
LUGO,1210.600000,19.800000,780.600000,3.600000,177.40000
ALBACETE,1402.500000,12.500000,782.500000,6.000000,174.50000


In [37]:
df_coef_provincia.sort_values('altitud', ascending=False).head(10)

,altitud,distancia,desnivel,pendiente,coeficiente
provincia,,,,,
TERUEL,1814.000000,14.800000,733.200000,5.200000,172.400000
SEGOVIA,1784.500000,9.500000,595.000000,6.000000,148.500000
ZAMORA,1770.000000,15.666667,751.666667,4.666667,143.000000
LLEIDA,1687.818182,13.272727,699.000000,4.909091,143.272727
ALMERÍA,1670.875000,18.750000,1001.375000,4.875000,216.375000
MADRID,1626.100000,10.800000,590.600000,4.700000,119.300000
PALENCIA,1484.333333,9.000000,454.333333,4.666667,87.333333
JAÉN,1464.000000,12.750000,838.000000,6.250000,210.500000
ÁVILA,1437.272727,10.454545,575.727273,5.090909,103.000000
